In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [3]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]

llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../graphfleet/output/20240829-184001/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Total report count: 280
Report count after filtering by community level 2: 221


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,278,# LATS: Enhancing Language Model Performance t...,3,9.5,LATS: Enhancing Language Model Performance thr...,The rating is high due to the comprehensive an...,The community revolves around the LATS (Langua...,[{'explanation': 'LATS (Language Agent Tree Se...,"{\n ""title"": ""LATS: Enhancing Language Mode...",d208e1b8-576f-4968-8c1b-92cb0fcb59d4
1,279,# RL-Based Training and Human Performance in W...,3,8.5,RL-Based Training and Human Performance in Web...,The rating is high due to the significant insi...,The community centers around RL-based training...,[{'explanation': 'RL-based training is a centr...,"{\n ""title"": ""RL-Based Training and Human P...",9c6a08a4-98c1-4cb6-bec6-52875abd78bf
2,145,# Monte Carlo Tree Search (MCTS) and its Algor...,2,9.0,Monte Carlo Tree Search (MCTS) and its Algorit...,The rating is high due to the significant impa...,The community revolves around the Monte Carlo ...,[{'explanation': 'Monte Carlo Tree Search (MCT...,"{\n ""title"": ""Monte Carlo Tree Search (MCTS...",c1d17e27-378b-4d79-bcea-d27df46432d5
3,146,# Search Algorithms in LATS and ADAS\n\nThe co...,2,9.0,Search Algorithms in LATS and ADAS,The rating is high due to the critical role th...,The community revolves around various search a...,[{'explanation': 'Search algorithms such as DF...,"{\n ""title"": ""Search Algorithms in LATS and...",471b2b23-dd3e-4851-a064-e3ac36c653b8
4,147,# Tree-Based Search and Key Contributors\n\nTh...,2,9.0,Tree-Based Search and Key Contributors,The impact rating is high due to the significa...,The community revolves around the Tree-Based S...,[{'explanation': 'Tree-Based Search is the cen...,"{\n ""title"": ""Tree-Based Search and Key Con...",200708bc-44e2-4cfd-9a57-ce82e132ee55


#### Build global context based on community reports

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [13]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=True,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",
    # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
result = await search_engine.asearch(
    "Write an example of multi-agent system that incorporates the tree search algorithm? and GraphRAG?Show me an agent that can help me create a knowledge graph"
)

print(result.response)

### Multi-Agent System Incorporating Tree Search Algorithm and GraphRAG

A multi-agent system that incorporates both the tree search algorithm and GraphRAG can be designed to enhance decision-making, planning, and knowledge graph construction. Below is an example of such a system:

#### System Overview

The system consists of multiple agents, each specialized in different tasks such as data retrieval, decision-making, and knowledge graph construction. The integration of tree search algorithms and GraphRAG ensures that the system can efficiently explore decision paths and generate comprehensive knowledge graphs.

#### Key Components

1. **Tree Search Agents**:
    - **Language Agent Tree Search (LATS)**: This agent uses Monte Carlo Tree Search (MCTS) to explore decision trees and optimize outcomes. LATS integrates language models and self-reflections to enhance decision-making and problem-solving capabilities. It is particularly effective in domains like programming, interactive questio

In [10]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,81,GPT-4 and its Benchmarking Ecosystem,1.00,# GPT-4 and its Benchmarking Ecosystem\n\nThe ...,9.5
1,218,Meta Agent Search and ADAS Framework,1.00,# Meta Agent Search and ADAS Framework\n\nThe ...,9.5
2,118,ARC Challenge and Transformation Rules,0.20,# ARC Challenge and Transformation Rules\n\nTh...,9.0
3,174,Mistral-7B-Instruct and Orca-3 Performance Com...,0.20,# Mistral-7B-Instruct and Orca-3 Performance C...,8.5
4,61,Neo4J and NaLLM in Knowledge Graph Creation,0.15,# Neo4J and NaLLM in Knowledge Graph Creation\...,9.0
...,...,...,...,...,...
216,171,Mostly Basic Programming Problems (MBPP) and A...,0.05,# Mostly Basic Programming Problems (MBPP) and...,8.5
217,65,Library Reconstruction and Nutritional APIs,0.05,# Library Reconstruction and Nutritional APIs\...,8.5
218,80,ACI-BENCH and its Contributors,0.10,# ACI-BENCH and its Contributors\n\nThe commun...,9.0
219,84,Medical Question-Answering Datasets and GPT Va...,0.10,# Medical Question-Answering Datasets and GPT ...,9.0


In [11]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 15. LLM tokens: 171906
